## Script to create zonal mean of polygons (in shp file)

In [1]:
import os
from time import time
import xarray as xr
import rioxarray
import fiona
import rasterio
import rasterio.mask as msk
import numpy as np
from datetime import date, datetime
from dateutil.rrule import rrule, DAILY
import pandas as pd

#### Function to perform zonal mean on polygon and output data in csv file

In [2]:
def zonal_stats(shape_path, infile, parameter,
                dict={"Year": [], "DOY": [], "parameter": [], "YYYYDOY": [], 'ID': [], 'name': [], 'mean': []}):
    # === Zonal Stats ===
    with fiona.open(shape_path, 'r') as shp:
        features = [feature for feature in shp]
        with rasterio.open(infile) as src:
            for f in features:
                src_shp = [f['geometry']]
                outimage, out_transform = msk.mask(src, src_shp, crop=True)
                ws_id = f['id']
                ws_name = f['properties']['Name']
                ws_mean = np.nanmean(outimage)
                print(ws_mean)
                ws_year = infile.split('.')[0][-7:-3]
                ws_doy = infile.split('.')[0][-3:]
                dict['name'].append(ws_name)
                dict['mean'].append(ws_mean)
                dict['Year'].append(ws_year)
                dict['DOY'].append(ws_doy)
                dict['YYYYDOY'].append('{}_{}'.format(ws_year,ws_doy))
                dict['parameter'].append(parameter)
                dict['ID'].append(ws_id)
    return dict

In [3]:
path_mos_dat = 's3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer'  # /year
path_mos_ppt = 's3://dev-et-data/in/USA_data/precip_gridmet_tiffs' # /year
shape_path = 'Small Watersheds 128_Aggregated.shp'
csv_output = '.'  #"." represent the current dir in Linux or type '/home/jupyter-kagone/postprocess'
parameter = 'prec'

a = datetime(2000, 1, 1)
b = datetime(2000, 1, 3)
#b = datetime(2019, 12, 31)

for dt in rrule(DAILY, dtstart=a, until=b):
    #print(dt.strftime("%Y-%m-%d")) # this returns string
    year = dt.strftime("%Y")
    month = dt.strftime("%m")
    day = dt.strftime("%d")
    doy = dt.timetuple().tm_yday
    print('Day: {} = {}-{}-{}'.format(doy, year, month, day))
        
    in_file = os.path.join(path_mos_ppt, year, '{}_{}{:03d}.tif'.format(parameter, year, doy))
    print(in_file)

    if dt == a:
        outdict = zonal_stats(shape_path, in_file, parameter)
    elif dt < b:
        outdict = zonal_stats(shape_path, in_file, parameter, dict=outdict)
    else:
        finaldict = zonal_stats(shape_path, in_file, parameter, dict=outdict)

#csvdf = pd.DataFrame(finaldict, columns=['Year', 'DOY', 'YYYYDOY', 'parameter', 'ID',  'name', 'mean'])
csvdf = pd.DataFrame(finaldict, columns=['YYYYDOY', 'ID',  'mean'])
print(csvdf)
#csvdf = csvdf.pivot(index=None, columns='ID', values='mean')
csvdf = pd.pivot_table(csvdf, values='mean', index='YYYYDOY', columns='ID').reset_index()

csvdf[['YYYY','DOY']] = csvdf.YYYYDOY.apply(lambda x: pd.Series(str(x).split("_"))) 

shp_list = list(range(53))
listf = []
listf.append('YYYYDOY')
listf.append('YYYY')
listf.append('DOY')
for elem in shp_list:
    listf.append('{}'.format(elem))
#print(listf)
csvdf = csvdf[listf]


#csvdf.to_csv(os.path.join(csv_output, '{}_zonalmean_2000_2019a.csv'.format(parameter)))
print('done')

Day: 1 = 2000-01-01
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000001.tif
20730.14285714286
21844.666666666668
18021.85
24575.25
24575.25
14043.028571428571
25485.444444444445
21844.666666666668
27647.15625
27305.833333333332
22936.9
20752.433333333334
21844.725
24575.2625
17943.833333333332
25257.895833333332
22936.9
23209.958333333332
21298.55
24029.133333333335
23830.545454545456
21298.55
21298.55
26623.1875
19660.2
24965.333333333332
20934.472222222223
24575.25
24029.133333333335
21844.666666666668
22117.725
20479.375
22936.9
21844.666666666668
24575.25
24575.25
21844.666666666668
32767.0
23405.0
17475.733333333334
27851.95
25745.5
26213.6
24575.25
20479.375
22936.9
26623.1875
20970.88
21844.666666666668
24029.133333333335
21844.666666666668
20479.375
19660.2
Day: 2 = 2000-01-02
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000002.tif
20730.14285714286
21844.666666666668
18022.305
24575.5
24575.525
14043.57619047619
25485.699999999997
21845.05416666